# Phrase (collocation) Detection Solution

###### Author: Alex Sherman | alsherman@deloitte.com

#### Agenda
1. SpaCy POS phrases
2. Gensim Phrases and Phraser

In [23]:
from collections import defaultdict
from configparser import ConfigParser, ExtendedInterpolation

from IPython.core.display import display, HTML
import pandas as pd
import spacy
from spacy import displacy
from spacy.matcher import Matcher
from spacy.matcher import PhraseMatcher
from spacy.lang.en.stop_words import STOP_WORDS
from sqlalchemy import create_engine

pd.set_option('display.max_colwidth', 100)

In [24]:
# configuration for data, acronyms, and gensim paths
config = ConfigParser(interpolation=ExtendedInterpolation())
config.read('../../config.ini')

DB_PATH = config['DATABASES']['PROJECT_DB_PATH']
MATCHED_TEXT_PATH = config['NLP']['MATCHED_TEXT_PATH']
CLEANED_TEXT_PATH = config['NLP']['CLEANED_TEXT_PATH']
GENSIM_DICTIONARY_PATH = config['NLP']['GENSIM_DICTIONARY_PATH']
GENSIM_CORPUS_PATH = config['NLP']['GENSIM_CORPUS_PATH']

#### Load data

In [25]:
# count # of documents
engine = create_engine(DB_PATH)
pd.read_sql("SELECT COUNT(*) FROM pubmed ", con=engine)

,COUNT(*)
0,500000


In [26]:
df = pd.read_sql("SELECT * FROM pubmed LIMIT 500", con=engine)

# filter to relevant sections
df.head()

,index,text
0,0,acid base diagrams
1,1,immunising against receptors for antigen
2,2,immune complexes in rheumatic disease
3,3,loss of hla antigens associated with hormonal state
4,4,hypocalcaemia after thyroidectomy


In [27]:
# store section matches in list
text = [text for text in df['text']]

# review first sentence of a section match
len(text)

500

### SpaCy - Preprocessing

In [28]:
%%time

# load spacy nlp model
# use 'en' if you don't have the lg model
nlp = spacy.load('en_core_web_lg')

Wall time: 31.1 s


###### collect sentences using SpaCy matcher

In [29]:
def collect_phrase_model_sents(matcher, doc, i, matches):
    # identify matching spans (phrases)
    match_id, start, end = matches[i]
    
    # get sentence with matched term
    sent = doc[start:end].sent.text
    
    # collect matching (cleaned) sents
    matched_sents.append(sent)

##### match sentences

https://explosion.ai/demos/matcher

In [30]:
%%time

# remove false statement below to run code
if 1 == 1:
    # match sentences with the word disease or disorder
    matched_sents = []
    pattern = [[{'LOWER': 'disease'}], [{'LOWER': 'disorder'}]]

    matcher = Matcher(nlp.vocab)

    # use *patterns to add more than one pattern at once
    matcher.add('disease_disorder', collect_phrase_model_sents, *pattern)

    for doc in nlp.pipe(text, disable=['tagger','ner']):    
        matcher(doc)

Wall time: 14.4 s


In [31]:
print('Number of matches: {} \n'.format(len(matched_sents)))

print('Example Match:')
print(matched_sents[0])

Number of matches: 12 

Example Match:
immune complexes in rheumatic disease


##### Export matched text to avoid repeating processing

In [32]:
# view path to matched text
MATCHED_TEXT_PATH

'C:\\Users\\alsherman\\Desktop\\NLP\\nlp_practicum_cohort2\\raw_data/cleaned_text/matched_text.txt'

In [33]:
# uncomment below to write the matched text to a .txt file for later use 

#with open(MATCHED_TEXT_PATH, 'w') as f:
#    for line in matched_sents:
#        line += '\n'
#        line = line.encode('ascii', errors='ignore').decode('ascii') 
#        f.write(line)

In [34]:
# read matched text
with open(MATCHED_TEXT_PATH, 'r') as f:
    matched_sents_full = [line.strip() for line in f.readlines()]

In [35]:
# store all matched sentences in a dataframe
matches_df = pd.DataFrame(matched_sents_full, columns=['sentences'])

# remove duplicates
matches_df = matches_df.drop_duplicates()

# recreate matched_sents (since it takes so long to create on its own)
matched_sents = [sent[0].split() for sent in matches_df.values]

# view matches
matches_df.head()

,sentences
0,immune complexes in rheumatic disease
1,gluten and lymphocytes in coeliac disease
2,this activity correlated best with the severity and duration of the disease rather than with gam...
3,evidence exists that they control some of the peripheral manifestations of the disease including...
4,the finding of additional substitute cf antigens for the hrvl agent may have implications in the...


### Use SpaCy part of speech (POS) to create phrases

In [36]:
# combine the matched sentence tokens and parse it with SpaCy
text = ' '.join(matched_sents[2])
text

'this activity correlated best with the severity and duration of the disease rather than with gamma globulin or total protein concentrations'

##### Determine which NLP components can be disabled

In [37]:
def view_pos(doc, n_tokens=5):
    """ print SpaCy POS information about each token in a provided document """
    print('{:15} | {:10} | {:10} | {:30}'.format('TOKEN','POS','DEP_','LEFTS'))
    for token in doc[0:n_tokens]:
        print('{:15} | {:10} | {:10} | {:30}'.format(
            token.text, token.head.pos_,token.dep_, str([t.text for t in token.lefts])))

In [38]:
# observe which part of speech (pos) attributes are disabled by named entity recognition (ner)
pos_doc = nlp(text, disable=['ner'])
view_pos(pos_doc)

TOKEN           | POS        | DEP_       | LEFTS                         
this            | NOUN       | det        | []                            
activity        | VERB       | nsubj      | ['this']                      
correlated      | VERB       | ROOT       | ['activity']                  
best            | VERB       | advmod     | []                            
with            | VERB       | prep       | []                            


In [39]:
# observe which part of speech (pos) attributes are disabled by parser
pos_doc = nlp(text, disable=['ner','parser'])
view_pos(pos_doc)

TOKEN           | POS        | DEP_       | LEFTS                         
this            | DET        |            | []                            
activity        | NOUN       |            | []                            
correlated      | VERB       |            | []                            
best            | ADV        |            | []                            
with            | ADP        |            | []                            


In [40]:
# observe which part of speech (pos) attributes are disabled by tagger
pos_doc = nlp(text, disable=['ner','tagger'])
view_pos(pos_doc, n_tokens=5)

TOKEN           | POS        | DEP_       | LEFTS                         
this            |            | det        | []                            
activity        |            | nsubj      | ['this']                      
correlated      |            | ROOT       | ['activity']                  
best            |            | advmod     | []                            
with            |            | prep       | []                            


In [41]:
# use explain to define any token.dep_ attributes
spacy.explain('dobj')

'direct object'

In [42]:
dependency_parsing_labels_url = 'https://spacy.io/api/annotation#dependency-parsing'
iframe = '<iframe src={} width=1000 height=400></iframe>'.format(dependency_parsing_labels_url)
HTML(iframe)

##### Extract phrases by identifying tokens describing an object

In [43]:
# add stop words to SpaCy
# this enables the .is_stop attribute with common stop words
from spacy.lang.en.stop_words import STOP_WORDS

for word in STOP_WORDS:
    lex = nlp.vocab[word]
    lex.is_stop = True

In [44]:
displacy_doc = nlp(text)

# show visualization in Jupyter Notebook
displacy.render(docs=displacy_doc, style='dep', jupyter=True)

In [45]:
def create_pos_phrases(doc):

    phrases = [] 

    doc = nlp(doc, disable=['ner','tagger'])
    for token in doc:
        direct_object = 'obj' in token.dep_
        if direct_object:
            # find any dependent terms to the left of (preceeding) the object
            for left_term in (t.text for t in token.lefts if not t.is_stop):
                # combine the dependent term and object, separated by an underscore
                # e.g. travel agency ==> travel_agency
                phrase = '{}_{}'.format(left_term,token.text)
                phrases.append(phrase)
    
    # convert list of distinct phrases into a sentence
    return ' '.join(set(phrases))

In [46]:
# review data
matches_df.head()

,sentences
0,immune complexes in rheumatic disease
1,gluten and lymphocytes in coeliac disease
2,this activity correlated best with the severity and duration of the disease rather than with gam...
3,evidence exists that they control some of the peripheral manifestations of the disease including...
4,the finding of additional substitute cf antigens for the hrvl agent may have implications in the...


In [47]:
%%time

for sent in matched_sents_full[0:5]:
    print(create_pos_phrases(sent))

rheumatic_disease
coeliac_disease
gamma_globulin
peripheral_manifestations cardiac_output palpitations_tachycardia thyrotoxic_process underlying_process
additional_antigens human_disease hrvl_agent cf_antigens
Wall time: 225 ms


In [48]:
%%time

# apply the custom function to every element in the dataframe
matches_df[0:5].sentences.apply(create_pos_phrases)

Wall time: 218 ms


0                                                                                      rheumatic_disease
1                                                                                        coeliac_disease
2                                                                                         gamma_globulin
3    peripheral_manifestations cardiac_output palpitations_tachycardia thyrotoxic_process underlying_...
4                                               additional_antigens human_disease hrvl_agent cf_antigens
Name: sentences, dtype: object

##### Pandas Apply

apply is an efficient and fast approach to 'apply' a function to every element in a row. applymap does the same to every element in the entire dataframe (e.g. convert all ints to floats)

Example: https://chrisalbon.com/python/data_wrangling/pandas_apply_operations_to_dataframes/

In [49]:
# create a small dataframe with example data
example_data = {'col1':range(0,3),'col2':range(3,6)}
test_df = pd.DataFrame(example_data)
test_df

,col1,col2
0,0,3
1,1,4
2,2,5


In [50]:
# apply a built-in function to each element in a column
test_df['col1'].apply(float)

0    0.0
1    1.0
2    2.0
Name: col1, dtype: float64

In [51]:
# apply a custom function to every element in a column
def add_five(row):
    return row + 5

test_df['col1'].apply(add_five)

0    5
1    6
2    7
Name: col1, dtype: int64

In [52]:
# apply an annonomous function to every element in a column
test_df['col1'].apply(lambda x: x+5)

0    5
1    6
2    7
Name: col1, dtype: int64

In [53]:
# apply a built-in function to every element in a dataframe 
test_df.applymap(float)  # applymap

,col1,col2
0,0.0,3.0
1,1.0,4.0
2,2.0,5.0


In [54]:
# create a new empty column
matches_df['create_pos_phrases'] = ''

# apply the custom function to every element in the dataframe
matches_df.loc[0:5, 'create_pos_phrases'] = matches_df[0:5].sentences.apply(create_pos_phrases)
matches_df.head()

,sentences,create_pos_phrases
0,immune complexes in rheumatic disease,rheumatic_disease
1,gluten and lymphocytes in coeliac disease,coeliac_disease
2,this activity correlated best with the severity and duration of the disease rather than with gam...,gamma_globulin
3,evidence exists that they control some of the peripheral manifestations of the disease including...,peripheral_manifestations cardiac_output palpitations_tachycardia thyrotoxic_process underlying_...
4,the finding of additional substitute cf antigens for the hrvl agent may have implications in the...,additional_antigens human_disease hrvl_agent cf_antigens


### Collocations

"A collocation is an expression consisting of two or more words that
correspond to some conventional way of saying things. Or in the words
of Firth (1957: 181): “Collocations of a given word are statements of the
habitual or customary places of that word.” Collocations include noun
phrases like strong tea and weapons of mass destruction, phrasal verbs like
to make up, and other stock phrases like the rich and powerful. Particularly
interesting are the subtle and not-easily-explainable patterns of word usage
that native speakers all know: why we say a stiff breeze but not a stiff wind
(while either a strong breeze or a strong wind is okay), or why we speak of
broad daylight (but not bright daylight or narrow darkness)



There are actually different definitions of the notion of collocation. Some
authors in the computational and statistical literature define a collocation
as two or more consecutive words with a special behavior, for example
Choueka (1988):
[A collocation is defined as] a sequence of two or more consecutive
words, that has characteristics of a syntactic and semantic
unit, and whose exact and unambiguous meaning or connotation
cannot be derived directly from the meaning or connotation of its
components. In most linguistically oriented research, a phrase
can be a collocation even if it is not consecutive (as in the example knock
. . . door). The following criteria are typical of linguistic treatments of collocations:

**Non-compositionality**: The meaning of a collocation is not a straightforward
composition of the meanings of its parts. Either the meaning
is completely different from the free combination (as in the case of idioms
like kick the bucket) or there is a connotation or added element of
meaning that cannot be predicted from the parts. For example, white
wine, white hair and white woman all refer to slightly different colors, so
we can regard them as collocations. 

**Non-substitutability**: We cannot substitute near-synonyms for the
components of a colloction. For example, we can’t say yellow wine
instead of white wine even though yellow is as good a description of the
color of white wine as white is (it is kind of a yellowish white).

**Non-modifiability**: Many collocations cannot be freely modified with
additional lexical material or through grammatical transformations.
This is especially true for frozen expressions like idioms. For example,
we can’t modify frog in to get a frog in one’s throat into to get an ugly
frog in one’s throat although usually nouns like frog can be modified by
adjectives like ugly. Similarly, going from singular to plural can make
an idiom ill-formed, for example in people as poor as church mice."

SOURCE: https://nlp.stanford.edu/fsnlp/promo/colloc.pdf

### Exercise

Create a function that returns a window of size n over a given sentence. 

For the sentence **'rather than pay the fee'** return the following if the window is n=3:
- ['rather', 'than', 'pay'],
- ['than','pay','the']
- ['pay', 'the','fee']
- ...


In [55]:
# example sentence
sent = ' '.join(matches_df['sentences'][0:1]).split()
print(sent)

['immune', 'complexes', 'in', 'rheumatic', 'disease']


In [56]:
def create_sentence_windows(sentence, n=3):
    "create a sliding window over the n terms in a list of terms"
        
    # create a window on the first n terms by slicing the sentence into the first n terms
    window = sentence[0:n]
    
    # create a list to store all windows
    # add the first window that was created above
    sentence_windows = [window]

    # iterate through the rest of the terms of the sentence
    # e.g. if n=3, then create a new window with terms 2 to 4
    for term in sentence[n:]:
        # remove the first terms of the window and add the next term from the sentence
        window = window[1:] + [term]
        # add the updated window to the master list
        sentence_windows.append(window)

    return sentence_windows

# execute the function
sentence_window = create_sentence_windows(sent, n=3)
# view the first few results
sentence_window[0:5]

[['immune', 'complexes', 'in'],
 ['complexes', 'in', 'rheumatic'],
 ['in', 'rheumatic', 'disease']]

In [57]:
# execute the function for all sentences

# create a list to store all windows
sentence_window = []

for sent in matches_df['sentences']:
    # convert the sentence string into a list of terms
    sent = sent.split()
    
    # create the sentence windows and append to the sentence_windows list
    windows = create_sentence_windows(sent, n=3)
    
    # add each window to the sentence_window list
    # iterate through windows to make each item in sentence window a window, not a list of windows
    for window in windows:
        sentence_window.append(window)

# view the first five results
sentence_window[0:5]

[['immune', 'complexes', 'in'],
 ['complexes', 'in', 'rheumatic'],
 ['in', 'rheumatic', 'disease'],
 ['gluten', 'and', 'lymphocytes'],
 ['and', 'lymphocytes', 'in']]

In [58]:
from itertools import combinations
from collections import defaultdict

# create a defaultdict to keep track of common phrases
window_count = defaultdict(int)

for sent in sentence_window:
    # remove stop words
    sentence = [term for term in sent if term not in STOP_WORDS]
    
    # create a combination of terms
    # e.g. (rather, than, pay) --> (rather,than), (than,pay), (rather,pay)
    for combo in combinations(sentence, 2):
        # convert the tuple to a term
        # e.g. (rather, than) --> 'rather_than'
        phrase = '_'.join(combo)
        
        # increment the count for the term each time it appears to identify the most common terms
        window_count[phrase] += 1

# sort to view the most common terms
# the key (lambda x: x[1]) sorts by the count
sorted(window_count.items(), key=lambda x: x[1], reverse=True)[0:20]

[('disease_virus', 1118),
 ('heart_disease', 1044),
 ('hodgkin_disease', 673),
 ('mouth_disease', 662),
 ('marek_disease', 527),
 ('graves_disease', 497),
 ('liver_disease', 482),
 ('coronary_heart', 399),
 ('coronary_disease', 388),
 ('foot_mouth', 336),
 ('crohn_disease', 306),
 ('ischemic_heart', 274),
 ('paget_disease', 265),
 ('cushing_disease', 256),
 ('newcastle_disease', 253),
 ('chronic_disease', 247),
 ('malignant_disease', 239),
 ('author_transl', 211),
 ('respiratory_disease', 207),
 ('mouth_virus', 204)]

### Phrase (collocation) Detection

Phrase modeling is another approach to learning combinations of tokens that together represent meaningful multi-word concepts. We can develop phrase models by looping over the the words in our reviews and looking for words that co-occur (i.e., appear one after another) together much more frequently than you would expect them to by random chance. The formula our phrase models will use to determine whether two tokens $A$ and $B$ constitute a phrase is:

$$\frac{count(A\ B) - count_{min}}{count(A) * count(B)} > threshold$$

- $count(A\ B)$ is the number of times the tokens $A\ B$ appear in the corpus in order
- $count_{min}$ is a user-defined parameter to ensure that accepted phrases occur a minimum number of times
- $count(A)$ is the number of times token $A$ appears in the corpus
- $count(B)$ is the number of times token $B$ appears in the corpus
- $threshold$ is a user-defined parameter to control how strong of a relationship between two tokens the model requires before accepting them as a phrase

Once our phrase model has been trained on our corpus, we can apply it to new text. When our model encounters two tokens in new text that identifies as a phrase, it will merge the two into a single new token.

Phrase modeling is superficially similar to named entity detection in that you would expect named entities to become phrases in the model (so new york would become new_york). But you would also expect multi-word expressions that represent common concepts, but aren't specifically named entities (such as happy hour) to also become phrases in the model.

We turn to the indispensible gensim library to help us with phrase modeling — the Phrases class in particular.

SOURCE: 
- https://github.com/skipgram/modern-nlp-in-python/blob/master/executable/Modern_NLP_in_Python.ipynb
- https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf

##### Scikit-learn API for Gensim

In [59]:
print(matched_sents[0:10])

[['immune', 'complexes', 'in', 'rheumatic', 'disease'], ['gluten', 'and', 'lymphocytes', 'in', 'coeliac', 'disease'], ['this', 'activity', 'correlated', 'best', 'with', 'the', 'severity', 'and', 'duration', 'of', 'the', 'disease', 'rather', 'than', 'with', 'gamma', 'globulin', 'or', 'total', 'protein', 'concentrations'], ['evidence', 'exists', 'that', 'they', 'control', 'some', 'of', 'the', 'peripheral', 'manifestations', 'of', 'the', 'disease', 'including', 'nervousness', 'palpitations', 'tachycardia', 'increased', 'cardiac', 'output', 'and', 'tremor', 'but', 'they', 'do', 'not', 'appear', 'to', 'affect', 'the', 'underlying', 'thyrotoxic', 'process', 'itself'], ['the', 'finding', 'of', 'additional', 'substitute', 'cf', 'antigens', 'for', 'the', 'hrvl', 'agent', 'may', 'have', 'implications', 'in', 'the', 'against', 'human', 'disease'], ['most', 'of', 'them', 'were', 'found', 'in', 'patients', 'with', 'digestive', 'tract', 'disease', 'essentially', 'colonic', 'cancer'], ['latency', 'in

In [60]:
from gensim.sklearn_api.phrases import PhrasesTransformer

sklearn_phrases = PhrasesTransformer(min_count=5, threshold=5)
sklearn_phrases.fit(matched_sents)

C:\Users\alsherman\AppData\Local\Continuum\anaconda3\envs\guild\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


PhrasesTransformer(delimiter=b'_', max_vocab_size=40000000, min_count=5,
          progress_per=10000, scoring='default', threshold=5)

In [61]:
# review phrase matches
phrases = []
for terms in sklearn_phrases.transform(matched_sents):
    for term in terms:
        if term.count('_') >= 1:
            phrases.append(term)

print(set(phrases))

C:\Users\alsherman\AppData\Local\Continuum\anaconda3\envs\guild\lib\site-packages\gensim\models\phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


{'benign_course', 'dermatitis_herpetiformis', 'lymphoblastic_leukemia', 'hla_bw', 'contribute_to', 'an_adult', 'dominantly_inherited', 'post_operative', 'to_produce', 'bowen_disease', 'newcastle_disease', 'huntington_disease', 'studies_on', 'causative_agents', 'platelet_aggregation', 'tool_for', 'immune_system', 'cells_infected', 'virus_neutralizing', 'be_demonstrated', 'attributable_to', 'fibroblasts_from', 'followed_by', 'no_difference', 'determined_by', 'follow_up', 'with_ulcerative', 'corticosteroid_therapy', 'other_causes', 'been_reported', 'designed_to', 'venereal_disease', 'adult_form', 'seven_patients', 'wilson_disease', 'who_are', 'believe_that', 'most_effective', 'in_terms', 'double_blind', 'has_provided', 'alpha_chain', 'nucleic_acid', 'relation_to', 'no_correlation', 'high_levels', 'pathogenesis_of', 'accompanied_by', 'our_experience', 'body_weight', 'urinary_tract', 'whipple_disease', 'herpesvirus_mdhv', 'one_day', 'correlate_with', 'short_stature', 'not_yet', 'burkitt_lym

In [62]:
# create a list of stop words
from spacy.lang.en.stop_words import STOP_WORDS
common_terms = list(STOP_WORDS)

**common_terms:** optional list of “stop words” that won’t affect frequency count of expressions containing them.
- The common_terms parameter add a way to give special treatment to common terms (aka stop words) such that their presence between two words won’t prevent bigram detection. It allows to detect expressions like “bank of america” or “eye of the beholder”.


##### Gensim API
A more complex API, though it is faster and has better integration with other gensim components (e.g. Phraser)

In [63]:
from gensim.models.phrases import Phrases, Phraser

In [64]:
phrases = Phrases(
      matched_sents
    , common_terms=common_terms
    , min_count=5
    , threshold=5
    , scoring='default'
)

phrases

### Phrases Params

- **scoring:** specifies how potential phrases are scored for comparison to the threshold setting. scoring can be set with either a string that refers to a built-in scoring function, or with a function with the expected parameter names. Two built-in scoring functions are available by setting scoring to a string:

    - ‘default’: from “Efficient Estimaton of Word Representations in Vector Space” by Mikolov, et. al.: 
    
$$\frac{count(AB) - count_{min}}{count(A) * count(B)} * N > threshold$$
    

    - where N is the total vocabulary size.
    - Thus, it is easier to exceed the threshold when the two words occur together often or when the two words are rare (i.e. small product)

In [65]:
bigram = Phraser(phrases)

bigram

The phrases object still contains all the source text in memory. A gensim Phraser will remove this extra data to become smaller and somewhat faster than using the full Phrases model. To determine what data to remove, the Phraser ues the  results of the source model’s min_count, threshold, and scoring settings. (You can tamper with those & create a new Phraser to try other values.)

SOURCE: https://radimrehurek.com/gensim/models/phrases.html

In [66]:
def print_phrases(phraser, text_stream, num_underscores=2):
    """ identify phrases from a text stream by searching for terms that
        are separated by underscores and include at least num_underscores
    """
    
    phrases = []
    for terms in phraser[text_stream]:
        for term in terms:
            if term.count('_') >= num_underscores:
                phrases.append(term)
    print(set(phrases))

In [67]:
print_phrases(bigram, matched_sents)

{'caries_and_periodontal', 'patients_with_suspected', 'review_of_the_literature', 'risk_of_coronary', 'clinical_and_laboratory', 'transposition_of_the_great', 'children_and_adolescents', 'children_with_congenital', 'alpha_and_beta', 'role_in_the_pathogenesis', 'signs_and_symptoms', 'weeks_of_age', 'factor_in_the_pathogenesis', 'carcinoma_of_the_lung', 'early_in_the_course', 'period_of_time', 'patients_with_graves', 'england_and_wales', 'studies_were_carried', 'carcinoma_in_situ', 'center_for_disease', 'incidence_of_coronary', 'despite_the_fact', 'risk_of_developing', 'clinical_and_pathological', 'plays_an_important', 'herpesvirus_of_turkeys', 'follow_up_period', 'outbreaks_of_respiratory', 'play_an_important', 'days_after_infection', 'greater_than_or_equal', 'infected_with_marek', 'etiology_and_pathogenesis', 'disease_of_the_vulva', 'diseases_with_special', 'patients_with_advanced', 'report_of_case', 'months_of_age', 'nursing_of_patient', 'taken_into_account', 'years_of_age', 'presence

### Tri-gram phrase model

We can place the text from the first phrase model into another Phrases object to create n-term phrase models. We can repear this process multiple times.

In [68]:
phrases = Phrases(
      bigram[matched_sents]
    , common_terms=common_terms
    , min_count=1
    , threshold=1
)

trigram = Phraser(phrases)

print_phrases(trigram, bigram[matched_sents], num_underscores=3)

{'lung_cancer_and_chronic_obstructive', 'age_of_the_patient', 'involvement_of_the_leptomeninges', 'particles_of_foot_and_mouth', 'reported_in_the_present', 'infected_cells_author_transl', 'patients_with_lung_cancer', 'virus_was_not_detected', 'virus_in_cell_culture', 'prenatal_diagnosis_of_tay_sachs', 'stage_iii_and_iv', 'clinical_and_pathological_features', 'evaluation_of_the_effect', 'exposure_to_infectious_bursal', 'carcinoma_of_the_breast', 'years_after_the_initial', 'suggest_that_the_natural_history', 'myocardial_infarction_is_probably', 'management_of_herpetic_eye', 'complex_in_paget_disease', 'hla_antigens_in_patients', 'good_correlation_between_the_activity', 'mortality_from_all_causes', 'cases_of_poliomyelitis_like', 'vivo_and_in_vitro', 'metastatic_disease_at_the_time', 'rise_to_the_suspicion', 'vaccine_against_aujeszky_disease', 'disease_of_the_nipple', 'involved_in_the_pathogenesis_of_this_disease', 'histochemical_and_ultrastructural_study', 'effect_of_carbon_monoxide', 'pa

In [69]:
for doc_num in [0]:
    print('DOC NUMBER: {}\n'.format(doc_num))
    print('ORIGINAL SENTENT: {}\n'.format(' '.join(matched_sents[doc_num])))
    print('BIGRAM: {}\n'.format(' '.join(bigram[matched_sents[doc_num]])))
    print('TRIGRAM: {}'.format(' '.join(trigram[bigram[matched_sents[doc_num]]])))
    print()

DOC NUMBER: 0

ORIGINAL SENTENT: immune complexes in rheumatic disease

BIGRAM: immune_complexes in rheumatic disease

TRIGRAM: immune_complexes in rheumatic_disease



#### Export Cleaned Text

In [70]:
# write the cleaned text to a new file for later use

#with open(CLEANED_TEXT_PATH, 'w') as f:
#    for line in bigram[matched_sents]:
#        line = ' '.join(line) + '\n'
#        line = line.encode('ascii', errors='ignore').decode('ascii')
#        f.write(line)